In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [4]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('cgan')):
    os.mkdir('saved_models_{}'.format('cgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('cgan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import fashion_mnist,cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Conv2DTranspose
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import Concatenate, GaussianNoise,Activation
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(2 * 2 * 512, activation='relu',input_dim=self.latent_dim))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Reshape((2, 2, 512)))

        model.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))


        model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add( BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))


        model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()

        # Conv 1: 16x16x64
        model.add(Conv2D(64, kernel_size=5, strides=2, padding='same' ,input_shape=self.img_shape))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        # Conv 2:
        model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        # Conv 3:
        model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
        model.add( BatchNormalization(momentum=0.9))
        model.add( LeakyReLU(alpha=0.1))

        # Conv 4:
        model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add( LeakyReLU(alpha=0.1))
        model.summary()

        # FC
        model.add(Flatten())
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')
        labels = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))
        discriminator =model(img)

        # Concatenate
        merged_layer = Concatenate()([discriminator, labels])
        discriminator = Dense(512, activation='relu')(merged_layer)

        # Output
        discriminator = Dense(1, activation='sigmoid')(discriminator)

        return Model([img, label], discriminator)




    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, _) = cifar10.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                if global_step % sample_interval == 0:
                    self.mode_drop(epoch, global_step)
                    

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise,sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')

if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=50, batch_size=64, sample_interval=200)


Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
batch_normalization_10 (Batc (None, 8, 8, 128)         512       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 256)         819456    
__________

epoch:0 step:61 [D loss: 0.032550, acc.: 100.00%] [G loss: 4.321854]
epoch:0 step:62 [D loss: 0.041061, acc.: 100.00%] [G loss: 4.958233]
epoch:0 step:63 [D loss: 0.016504, acc.: 100.00%] [G loss: 4.675683]
epoch:0 step:64 [D loss: 0.040117, acc.: 100.00%] [G loss: 4.357265]
epoch:0 step:65 [D loss: 0.152232, acc.: 100.00%] [G loss: 6.951140]
epoch:0 step:66 [D loss: 0.116403, acc.: 93.75%] [G loss: 6.715162]
epoch:0 step:67 [D loss: 0.047150, acc.: 99.22%] [G loss: 5.231528]
epoch:0 step:68 [D loss: 0.250142, acc.: 89.84%] [G loss: 7.656771]
epoch:0 step:69 [D loss: 0.196772, acc.: 92.97%] [G loss: 6.381342]
epoch:0 step:70 [D loss: 0.096018, acc.: 96.09%] [G loss: 4.477276]
epoch:0 step:71 [D loss: 0.098724, acc.: 100.00%] [G loss: 5.651394]
epoch:0 step:72 [D loss: 0.012283, acc.: 100.00%] [G loss: 6.087322]
epoch:0 step:73 [D loss: 0.024550, acc.: 100.00%] [G loss: 5.240605]
epoch:0 step:74 [D loss: 0.054299, acc.: 100.00%] [G loss: 4.727752]
epoch:0 step:75 [D loss: 0.040376, acc.

epoch:0 step:183 [D loss: 0.075122, acc.: 99.22%] [G loss: 2.863273]
epoch:0 step:184 [D loss: 0.130491, acc.: 96.88%] [G loss: 4.953936]
epoch:0 step:185 [D loss: 0.198770, acc.: 91.41%] [G loss: 3.244072]
epoch:0 step:186 [D loss: 0.231380, acc.: 92.97%] [G loss: 6.257140]
epoch:0 step:187 [D loss: 1.014951, acc.: 54.69%] [G loss: 2.828717]
epoch:0 step:188 [D loss: 0.048003, acc.: 99.22%] [G loss: 4.090246]
epoch:0 step:189 [D loss: 0.052829, acc.: 100.00%] [G loss: 3.275062]
epoch:0 step:190 [D loss: 0.151636, acc.: 95.31%] [G loss: 1.439602]
epoch:0 step:191 [D loss: 0.303601, acc.: 89.84%] [G loss: 5.756842]
epoch:0 step:192 [D loss: 0.563518, acc.: 78.12%] [G loss: 2.644011]
epoch:0 step:193 [D loss: 0.268356, acc.: 89.06%] [G loss: 3.834568]
epoch:0 step:194 [D loss: 0.121962, acc.: 98.44%] [G loss: 3.821657]
epoch:0 step:195 [D loss: 0.231395, acc.: 93.75%] [G loss: 3.271947]
epoch:0 step:196 [D loss: 0.303338, acc.: 87.50%] [G loss: 5.878960]
epoch:0 step:197 [D loss: 0.45839

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

epoch:49 step:38516 [D loss: 0.015756, acc.: 99.22%] [G loss: 0.000080]
epoch:49 step:38517 [D loss: 0.018162, acc.: 100.00%] [G loss: 0.008140]
epoch:49 step:38518 [D loss: 0.000255, acc.: 100.00%] [G loss: 0.000373]
epoch:49 step:38519 [D loss: 0.003421, acc.: 100.00%] [G loss: 0.677755]
epoch:49 step:38520 [D loss: 0.026381, acc.: 100.00%] [G loss: 0.000353]
epoch:49 step:38521 [D loss: 0.007023, acc.: 100.00%] [G loss: 0.082370]
epoch:49 step:38522 [D loss: 0.021408, acc.: 99.22%] [G loss: 0.345224]
epoch:49 step:38523 [D loss: 0.025165, acc.: 99.22%] [G loss: 0.003837]
epoch:49 step:38524 [D loss: 0.002851, acc.: 100.00%] [G loss: 0.005132]
epoch:49 step:38525 [D loss: 0.003395, acc.: 100.00%] [G loss: 0.075203]
epoch:49 step:38526 [D loss: 0.000532, acc.: 100.00%] [G loss: 0.004320]
epoch:49 step:38527 [D loss: 0.094237, acc.: 96.88%] [G loss: 1.205240]
epoch:49 step:38528 [D loss: 0.203996, acc.: 93.75%] [G loss: 0.036863]
epoch:49 step:38529 [D loss: 0.056232, acc.: 98.44%] [G 

epoch:49 step:38631 [D loss: 0.001075, acc.: 100.00%] [G loss: 0.113475]
epoch:49 step:38632 [D loss: 0.196261, acc.: 95.31%] [G loss: 0.091463]
epoch:49 step:38633 [D loss: 0.006154, acc.: 100.00%] [G loss: 1.374814]
epoch:49 step:38634 [D loss: 0.072983, acc.: 96.88%] [G loss: 0.167130]
epoch:49 step:38635 [D loss: 0.010957, acc.: 100.00%] [G loss: 0.054562]
epoch:49 step:38636 [D loss: 0.002593, acc.: 100.00%] [G loss: 0.282827]
epoch:49 step:38637 [D loss: 0.034319, acc.: 99.22%] [G loss: 0.028366]
epoch:49 step:38638 [D loss: 0.005404, acc.: 100.00%] [G loss: 0.093534]
epoch:49 step:38639 [D loss: 0.057384, acc.: 100.00%] [G loss: 0.101142]
epoch:49 step:38640 [D loss: 0.001764, acc.: 100.00%] [G loss: 0.142621]
epoch:49 step:38641 [D loss: 0.002408, acc.: 100.00%] [G loss: 0.064063]
epoch:49 step:38642 [D loss: 0.002294, acc.: 100.00%] [G loss: 0.018720]
epoch:49 step:38643 [D loss: 0.019621, acc.: 100.00%] [G loss: 0.013691]
epoch:49 step:38644 [D loss: 0.067200, acc.: 97.66%] [

epoch:49 step:38747 [D loss: 0.000439, acc.: 100.00%] [G loss: 0.792529]
epoch:49 step:38748 [D loss: 0.001010, acc.: 100.00%] [G loss: 0.000931]
epoch:49 step:38749 [D loss: 0.001314, acc.: 100.00%] [G loss: 0.002687]
epoch:49 step:38750 [D loss: 0.001379, acc.: 100.00%] [G loss: 0.027264]
epoch:49 step:38751 [D loss: 0.003483, acc.: 100.00%] [G loss: 0.000541]
epoch:49 step:38752 [D loss: 0.000280, acc.: 100.00%] [G loss: 0.000824]
epoch:49 step:38753 [D loss: 0.000161, acc.: 100.00%] [G loss: 0.051811]
epoch:49 step:38754 [D loss: 0.000049, acc.: 100.00%] [G loss: 0.149017]
epoch:49 step:38755 [D loss: 0.000580, acc.: 100.00%] [G loss: 0.002873]
epoch:49 step:38756 [D loss: 0.000118, acc.: 100.00%] [G loss: 0.001360]
epoch:49 step:38757 [D loss: 0.000654, acc.: 100.00%] [G loss: 0.003417]
epoch:49 step:38758 [D loss: 0.000117, acc.: 100.00%] [G loss: 0.000569]
epoch:49 step:38759 [D loss: 0.000141, acc.: 100.00%] [G loss: 0.042535]
epoch:49 step:38760 [D loss: 0.004457, acc.: 100.00

epoch:49 step:38862 [D loss: 0.043182, acc.: 98.44%] [G loss: 0.252847]
epoch:49 step:38863 [D loss: 0.005272, acc.: 100.00%] [G loss: 0.003351]
epoch:49 step:38864 [D loss: 0.010746, acc.: 100.00%] [G loss: 0.001547]
epoch:49 step:38865 [D loss: 0.055651, acc.: 98.44%] [G loss: 0.038084]
epoch:49 step:38866 [D loss: 0.007281, acc.: 100.00%] [G loss: 0.096063]
epoch:49 step:38867 [D loss: 0.007307, acc.: 100.00%] [G loss: 0.198340]
epoch:49 step:38868 [D loss: 0.006914, acc.: 100.00%] [G loss: 0.083136]
epoch:49 step:38869 [D loss: 0.001277, acc.: 100.00%] [G loss: 0.147774]
epoch:49 step:38870 [D loss: 0.076696, acc.: 97.66%] [G loss: 0.000465]
epoch:49 step:38871 [D loss: 0.002478, acc.: 100.00%] [G loss: 0.003957]
epoch:49 step:38872 [D loss: 0.001775, acc.: 100.00%] [G loss: 0.008300]
epoch:49 step:38873 [D loss: 0.009301, acc.: 100.00%] [G loss: 0.000548]
epoch:49 step:38874 [D loss: 0.003532, acc.: 100.00%] [G loss: 0.006021]
epoch:49 step:38875 [D loss: 0.004022, acc.: 100.00%] 

epoch:49 step:38975 [D loss: 0.000322, acc.: 100.00%] [G loss: 0.015806]
epoch:49 step:38976 [D loss: 0.003531, acc.: 100.00%] [G loss: 0.000589]
epoch:49 step:38977 [D loss: 0.001148, acc.: 100.00%] [G loss: 0.000495]
epoch:49 step:38978 [D loss: 0.000278, acc.: 100.00%] [G loss: 0.000573]
epoch:49 step:38979 [D loss: 0.000250, acc.: 100.00%] [G loss: 0.001213]
epoch:49 step:38980 [D loss: 0.000220, acc.: 100.00%] [G loss: 0.000323]
epoch:49 step:38981 [D loss: 0.000047, acc.: 100.00%] [G loss: 0.000428]
epoch:49 step:38982 [D loss: 0.006526, acc.: 100.00%] [G loss: 0.001728]
epoch:49 step:38983 [D loss: 0.000269, acc.: 100.00%] [G loss: 0.000297]
epoch:49 step:38984 [D loss: 0.007382, acc.: 100.00%] [G loss: 0.000922]
epoch:49 step:38985 [D loss: 0.000756, acc.: 100.00%] [G loss: 0.000853]
epoch:49 step:38986 [D loss: 0.001542, acc.: 100.00%] [G loss: 0.001258]
epoch:49 step:38987 [D loss: 0.002390, acc.: 100.00%] [G loss: 0.000133]
epoch:49 step:38988 [D loss: 0.000335, acc.: 100.00